# YCQL Intermediate Development Lab

In this lab, you will learn about how to use indexes to optimize the data model, validate data, and improve the query performance. First, you will learn how to measure the performance of queries, then you will learn how to create indexes to make the queries more efficient and performant at scale.

## Setup steps
Here are the steps to setup this lab:
- Import the notebook variables
- Create the `db_ybu` database
- Import the sql scripts

### Import the notebook variables 

The following Python cell creates and stores variables that all the notebooks in this lab will use. You can view these variables in the Jupyter tab.

- To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.
- Verify the accuracy of the output values

In [ ]:
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_GITPOD_WORKSPACE_URL
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_NOTEBOOK_DIR
%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_DATA_DDL_FILE
%store -r MY_DATA_DML_FILE

Stored 'MY_DB_NAME' (str)
Stored 'MY_YB_PATH' (str)
Stored 'MY_GITPOD_WORKSPACE_URL' (NoneType)
Stored 'MY_HOST_IPv4_01' (str)
Stored 'MY_HOST_IPv4_02' (str)
Stored 'MY_HOST_IPv4_03' (str)
Stored 'MY_NOTEBOOK_DIR' (str)
Stored 'MY_TSERVER_WEBSERVER_PORT' (str)
Stored 'MY_NOTEBOOK_DATA_FOLDER' (str)


UsageError: Unknown variable 'MY_NOTEBOOK_UTILS_FOLDER'


## Implement the Data Model
This section will create and populate the keyspace with the tables and data that was described for the wish list data model discussed in the YCQL development module.

### Create Keyspace: db_ybu
Run the following cell to execute the following operations.
* Change into the Yugabyte directory that contains the ycqlsh script that will allow a connection to the YCQL shell. 
* Drop the keyspace, `db_ybu`, if it exists.
* Create the keyspace, `db_ybu`.  
* Connect to the `db_ybu` keyspace.
* Describe the keyspace, `db_ybu`, to verify the keyspace was created correctly.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # Import file path of Yugabyte and DB name
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# Drop keyspace
./bin/ycqlsh --execute "
  DROP KEYSPACE IF EXISTS "${DB_NAME}";
"
 
# Create keyspace
./bin/ycqlsh --execute "
  CREATE KEYSPACE "${DB_NAME}";
" 

# Connect to keyspace
./bin/ycqlsh --execute "
  USE "${DB_NAME}";
"

# List keyspaces, validate keyspace creation
./bin/ycqlsh --execute "
  DESCRIBE keyspaces
"

# List keyspaces, validate keyspace creation
./bin/ycqlsh --execute "
  DESCRIBE "${DB_NAME}"
"

In the preceeding cell, the YCQL shell was accessed using `ycqlsh` to create the YCQL shell to access the YCQL keyspace. Once connected YCQL shell, the DDL, DML, and shell commands can create, connect, and describe  keyspaces and tables. The `USE` keyword assigns the active keyspace. The `DESCRIBE` is used to list all the available keyspaces in the cluster as well as describe a particular keyspace to display the schema of the tables.

## Load DDL and Data from File
Load SQL statements from a script
This will load the sql files located in the data folder of the project's root directory.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_DATA_FOLDER" "$MY_DATA_DDL_FILE" "$MY_DATA_DML_FILE"   
# Wishlist
YB_PATH=${1}
DB_NAME=${2}
DATA_FOLDER=${3}
DATA_DDL_FILE=${4}
DATA_DML_FILE=${5}

WISHLIST_DDL_PATH=${DATA_FOLDER}/${DATA_DDL_FILE}
WISHLIST_DML_PATH=${DATA_FOLDER}/${DATA_DML_FILE}
echo $WISHLIST_DDL_PATH
echo $WISHLIST_DML_PATH
cd $YB_PATH

# DDL file
./bin/ycqlsh -k ${DB_NAME} -f ${WISHLIST_DDL_PATH} 
sleep 1;

# # DML file
./bin/ycqlsh -k ${DB_NAME} -f ${WISHLIST_DML_PATH} 
sleep 1;

# # Describe relations
./bin/ycqlsh --execute "
  DESCRIBE "${DB_NAME}"
"

Validate the data was loaded properly from the SQL scripts.

### Query Plans

Evaluate the different scan types that evaluate the efficiency of database operations.

In the following cell, you will evaluate a sequential scan.

In [ ]:
%%bash -s "$MY_YB_PATH"   
YB_PATH=${1}
cd $YB_PATH

# Query plan: Sequential Scan
./bin/ycqlsh --execute "
  EXPLAIN SELECT * FROM db_ybu.tbl_products_by_category;
"  

A query plan helps reveal the inner workings of a query to illustrate the numbers of operations required to satisfy the query.

The preceding query was determined to be a sequential scan in the statement, `Seq Scan on db_ybu.tbl_products_by_category`. 

A sequential scan reveals that the query required a read from every row in the primary table. This was expected since the query has no where predicate, therefore no false conditions to deny a row to render in the result set.

### Where clause predicate
Run the following cell to see if having a conditional expression in the `WHERE` clause affects the query plan.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  EXPLAIN SELECT 
    product_name, 
    product_id, 
    price, 
    description 
  FROM db_ybu.tbl_products_by_category 
  WHERE product_name='Hotdogs';
"    

#### Query plan
In the preceding cell, the `WHERE` clause on line #13, searches for all rows that contain the product name "Hotdogs". The resulting query plan reveals that this query needed to scan every row in the primary table to determine which rows contained the product name "Hotdogs".

### Aggregate Scan
Run the following cell to determine the query plan for an query using an aggregate function. This query plan reveals another type of scan called a Range scan. A range scan reveals that many rows values are needed to satisfy this query.

In [ ]:
%%bash -s "$MY_YB_PATH" 
YB_PATH=${1}
cd $YB_PATH

# Query Plan for Aggregate function
./bin/ycqlsh --execute "
  EXPLAIN SELECT SUM(quantity) as subtotal 
  FROM db_ybu.tbl_products_by_wishlist where wishlist_id=?;
"  

### Secondary Indexes 
In this section, you will create an index and determine how this affects the efficiency of a query by comparing the query plans.

Run the following cell to create and describe an index by describing the tbl_products_by_category table.

In [ ]:
%%bash -s "$MY_YB_PATH" 
YB_PATH=${1}
cd $YB_PATH

# Create a secondary index, product_name
./bin/ycqlsh --execute "
  CREATE INDEX idx_products_by_name 
  ON db_ybu.tbl_products_by_category (product_name) 
  INCLUDE (description);
"   

# Verfiy the index has been created for the tbl_products_by_category
./bin/ycqlsh --execute "
  DESC db_ybu.tbl_products_by_category
"

#### Verify the query plan for a secondary index
In the previous cell, a secondary index was created so that the product name can be efficiently queried on the tbl_products_by_category table. The DESC keyword is used to verify if the index was created correctly. 

> **Important:** The attributes in the primary key remain a part of the secondary index and can be retrieved from the index without an extra trip to the primary table, improving operational efficiency. 

The description attribute was added using the `INCLUDE` keyword to the secondary index to create a covering index.

Run the following cell to verify if adding a secondary index for the `product_name` can make the product_name query on the `tbl_products_by_category` more efficient.

In [ ]:
%%bash -s "$MY_YB_PATH"   # Sequential Scan
YB_PATH=${1}
cd $YB_PATH

# Expose the query plan for a covering index
./bin/ycqlsh --execute "
  EXPLAIN SELECT product_name, category, price, description, product_id 
  FROM db_ybu.tbl_products_by_category where product_name=?;
"  

#### Index Only Scan
By running the previous cell, you were able to determine that by adding a secondary index for the condition attribute, `product_name`, the query plan has changed from a sequential scan to an index only scan. This has improved this query's performance by orders of magnitude when accounting for large scale workloads. 

The `INCLUDE` clause plays an essential role in making queries more efficient and indexes more useful.  In the previous cell, the `product_name` was added to the index by using the `INCLUDE` clause. Now, the secondary index contains the values of the `product_name` attribute, creating a covering index, reducing a trip to the primary table, also known as the heap in PostgreSQL.

For example, if you had not added the `description` attribute in the `INCLUDE` clause, that would have made the query plan an index scan. This means that in our example, although the index was used to locate product_name, the primary table was still accessed to retrieve the data for the `description` column.

### Unique Indexes
You can disallow a column from having duplicate values by using a unique constraint as shown in the following cell.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE UNIQUE INDEX idx_unique_product_by_id 
  ON db_ybu.tbl_products_by_category(product_id)
  INCLUDE(description);
"  
  
./bin/ycqlsh --execute "
  DESC db_ybu.tbl_products_by_category
"  
# This statement will cause an error since duplicate values for the 
# product_id are not allowed, 87c7624a-4af5-4347-922d-ab43ab32476b.
./bin/ycqlsh --execute "
  INSERT INTO tbl_products_by_category (
    product_name, 
    description, 
    price, 
    category,
    product_id 
  ) VALUES (
    'Guard dogs',
    'Doberman Pinchers',
    643.99,
    'Security',
    87c7624a-4af5-4347-922d-ab43ab32476b
  );
"

#### Partial Indexes

Use a range partition to reduce the amount of data that requires scanning.
In this index, you will partition all products over $30 in the Office Supply category. 

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE INDEX ON db_ybu.tbl_products_by_category(price) INCLUDE (description, product_name) WHERE price > 30 and category = 'Office Supplies';
"

./bin/ycqlsh --execute "
  DESC db_ybu.tbl_products_by_category
"  

#### Collections
More complex data structures allow YCQL to store data sets that offers more flexibility in its data model capabilities. Very important when a trying to reduce the amount of tables that need to be queried.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Collection: Create a column with a list containing text elements
./bin/ycqlsh --execute "ALTER TABLE db_ybu.tbl_products_by_category ADD warehouse_ids LIST<TEXT>;"
./bin/ycqlsh --execute "DESC db_ybu.tbl_products_by_category"

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Collection: Create a column for a set containing text elements
./bin/ycqlsh --execute "ALTER TABLE db_ybu.tbl_products_by_category ADD tags SET<TEXT>;"
./bin/ycqlsh --execute "DESC db_ybu.tbl_products_by_category"

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Frozen Collection
./bin/ycqlsh --execute "
  ALTER TABLE db_ybu.tbl_products_by_category ADD store_locations FROZEN<LIST<TEXT>>;
"

./bin/ycqlsh --execute "
  DESC db_ybu.tbl_products_by_category
"

#### Frozen Collections
In the preceding cell, note that a frozen collection cannot have its elements updated. The collection's elements are immutable. The value can be dropped, but not manipulated.

#### JSON

JSONB is considered the best way to utilize complex data structures since in YCQL, JSONB is searchable. This is not true of the other collections in YCQL. Also note that collections can be used in JSON as well. Run the following cell and note the pattern necessary to write a JSON object to a table in YCQL.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  INSERT INTO db_ybu.tbl_products_by_category (
    category, 
    price, 
    product_id, 
    sku_details
  ) VALUES (
    'Grocery',
    9.99,
    6eb8d774-8b03-4457-a8e9-710339ca7165,
    '{
      "product_id": "6eb8d774-8b03-4457-a8e9-710339ca7165",
      "warehouse_sku": "8jk39d03-8b03-4457-a8e9-710339ca7165"
    }'
  );
"

./bin/ycqlsh --execute "
  SELECT * FROM db_ybu.tbl_products_by_category;
"

In the preceding cell, note the syntax necessary to insert a JSON object into the table. Outside the curly brackets are single quotes, where inside the brackets there are double quotes.

#### JSONB Index

JSONB is considered the best way to utilize complex data structures since in YCQL, JSONB is searchable.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE INDEX idx_sku_details_name ON db_ybu.tbl_products_by_category((sku_details->>'Name'));
"

./bin/ycqlsh --execute "
  DESC  db_ybu.tbl_products_by_category
"

./bin/ycqlsh --execute "
  EXPLAIN SELECT category, price, product_id FROM db_ybu.tbl_products_by_category WHERE (sku_details->>'Name')=?;
"

In the preceding cell, note the syntax that is required to create an index using a JSON key. Also note the syntax used to search by the key in a JSON object.

### Time to Live

YCQL offers data expiration. In the context of data modelling, removing "old" or deprecated data can improve database operational costs as well as storage costs.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

# Query Plan
./bin/ycqlsh --execute "
  CREATE TABLE db_ybu.tbl_todolists_by_user (
    user_id BIGINT, 
    todolist_name TEXT, 
    todolist_id UUID, 
    is_public BOOLEAN, 
    PRIMARY KEY((user_id), todolist_name)
  ) 
  WITH CLUSTERING ORDER BY (todolist_name DESC);
"

In the following cell, a row is added to the table with a time-to-live of 5 seconds. Once the time has expired, notice that the row is no longer in the table. This is a TTL scoped to the data row.

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH

./bin/ycqlsh --execute "
  INSERT INTO db_ybu.tbl_wishlists_by_user(
      user_id, 
      wishlist_id, 
      name, 
      is_public
    ) VALUES (
      'Mark', 
      'Grocery', 
      2a70494e-6b68-4739-b3e0-ff06aa0a2d67, 
      true
    ) 
    USING TTL 5;
  "  

In [ ]:
%%bash -s "$MY_YB_PATH"  
YB_PATH=${1}
cd $YB_PATH
   
./bin/ycqlsh --execute "SELECT * FROM db_ybu.tbl_todolists_by_user;"  

If the row is still visible, wait a few more seconds to run the preceding cell. This will verify that the row has expired as expected.

---
# All done!
In this lab, you completed the following:

- Setup
  - Created the `db_ybu` database with `ycqlsh`
  - Created tables and loaded data using DDL and DML scripts